In [1]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

DATA_PATH = "/kaggle/input/fyp-dataset/AffectNet/"
TOP_EMOTIONS = ["anger", "contempt", "disgust", "fear", "happy", "neutral", "sad", "surprise"]

TRAIN_SIZE = 0.80
NUM_CLASSES = len(TOP_EMOTIONS)

2024-03-19 16:39:55.108816: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 16:39:55.108931: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 16:39:55.265581: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Define input size
input_shape = (48, 48, 1)

In [2]:
# Define input size
input_shape = (48, 48, 1)

total_images = 0
for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        count = len(os.listdir(os.path.join(DATA_PATH, dir_)))
        print(f"{dir_} has {count} number of images")
        total_images += count

print(f"\ntotal images are {total_images}")

img_arr = []
img_label = []
label_to_text = {}
label = 0

for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        for f in os.listdir(os.path.join(DATA_PATH, dir_)):
            img = cv2.imread(os.path.join(DATA_PATH, dir_, f), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, input_shape[:2])  # Resize the image
            img_arr.append(np.expand_dims(img, axis=-1))
            img_label.append(label)
        print(f"loaded {dir_} images to numpy arrays...")
        label_to_text[label] = dir_
        label += 1

img_arr = np.array(img_arr)
img_label = np.array(img_label)
img_label = OneHotEncoder().fit_transform(img_label.reshape(-1, 1)).toarray()  # Convert to dense array

surprise has 1851 number of images
fear has 1839 number of images
neutral has 1880 number of images
sad has 1821 number of images
disgust has 1740 number of images
contempt has 1833 number of images
happy has 1862 number of images
anger has 1822 number of images

total images are 14648
loaded surprise images to numpy arrays...
loaded fear images to numpy arrays...
loaded neutral images to numpy arrays...
loaded sad images to numpy arrays...
loaded disgust images to numpy arrays...
loaded contempt images to numpy arrays...
loaded happy images to numpy arrays...
loaded anger images to numpy arrays...


In [3]:
# Normalize pixel values
img_arr = img_arr / 255.

X_train, X_test, y_train, y_test = train_test_split(img_arr, img_label,
                                                    shuffle=True, stratify=img_label,
                                                    train_size=TRAIN_SIZE, random_state=42)

def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
# Train and evaluate the model
print(f"Training model with input shape: {input_shape}")
model = create_cnn_model(input_shape, NUM_CLASSES)
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Training model with input shape: (48, 48, 1)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 28s 71ms/step - accuracy: 0.2281 - loss: 1.9283 - val_accuracy: 0.3836 - val_loss: 1.5705
Epoch 2/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 18s 48ms/step - accuracy: 0.4094 - loss: 1.5224 - val_accuracy: 0.4137 - val_loss: 1.4640
Epoch 3/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 21s 49ms/step - accuracy: 0.4524 - loss: 1.4001 - val_accuracy: 0.4502 - val_loss: 1.4225
Epoch 4/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 20s 49ms/step - accuracy: 0.4900 - loss: 1.3301 - val_accuracy: 0.4679 - val_loss: 1.3762
Epoch 5/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 20s 47ms/step - accuracy: 0.5267 - loss: 1.2462 - val_accuracy: 0.4761 - val_loss: 1.3696
Epoch 6/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 21s 48ms/step - accuracy: 0.5466 - loss: 1.1921 - val_accuracy: 0.4911 - val_loss: 1.3529
Epoch 7/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 20s 48ms/step - accuracy: 0.5743 - loss: 1.1419 - val_accuracy: 0.4983 - val_loss: 1.3374
Epoch 8/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 20s 47ms/step - accuracy: 0.5973 - loss: 1.0780 - 

# Define input size
input_shape = (64, 64, 1)

In [5]:
# Define input size
input_shape = (64, 64, 1)

total_images = 0
for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        count = len(os.listdir(os.path.join(DATA_PATH, dir_)))
        print(f"{dir_} has {count} number of images")
        total_images += count

print(f"\ntotal images are {total_images}")

img_arr = []
img_label = []
label_to_text = {}
label = 0

for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        for f in os.listdir(os.path.join(DATA_PATH, dir_)):
            img = cv2.imread(os.path.join(DATA_PATH, dir_, f), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, input_shape[:2])  # Resize the image
            img_arr.append(np.expand_dims(img, axis=-1))
            img_label.append(label)
        print(f"loaded {dir_} images to numpy arrays...")
        label_to_text[label] = dir_
        label += 1

img_arr = np.array(img_arr)
img_label = np.array(img_label)
img_label = OneHotEncoder().fit_transform(img_label.reshape(-1, 1)).toarray()  # Convert to dense array

surprise has 1851 number of images
fear has 1839 number of images
neutral has 1880 number of images
sad has 1821 number of images
disgust has 1740 number of images
contempt has 1833 number of images
happy has 1862 number of images
anger has 1822 number of images

total images are 14648
loaded surprise images to numpy arrays...
loaded fear images to numpy arrays...
loaded neutral images to numpy arrays...
loaded sad images to numpy arrays...
loaded disgust images to numpy arrays...
loaded contempt images to numpy arrays...
loaded happy images to numpy arrays...
loaded anger images to numpy arrays...


In [6]:
# Normalize pixel values
img_arr = img_arr / 255.

X_train, X_test, y_train, y_test = train_test_split(img_arr, img_label,
                                                    shuffle=True, stratify=img_label,
                                                    train_size=TRAIN_SIZE, random_state=42)

def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
# Train and evaluate the model
print(f"Training model with input shape: {input_shape}")
model = create_cnn_model(input_shape, NUM_CLASSES)
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Training model with input shape: (64, 64, 1)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 33s 86ms/step - accuracy: 0.2499 - loss: 1.8828 - val_accuracy: 0.3877 - val_loss: 1.5159
Epoch 2/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 32s 86ms/step - accuracy: 0.4370 - loss: 1.4500 - val_accuracy: 0.4608 - val_loss: 1.3833
Epoch 3/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 42s 88ms/step - accuracy: 0.4881 - loss: 1.3166 - val_accuracy: 0.4915 - val_loss: 1.3373
Epoch 4/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 42s 91ms/step - accuracy: 0.5376 - loss: 1.2257 - val_accuracy: 0.4816 - val_loss: 1.3632
Epoch 5/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 33s 90ms/step - accuracy: 0.5856 - loss: 1.0985 - val_accuracy: 0.5140 - val_loss: 1.2868
Epoch 6/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 31s 84ms/step - accuracy: 0.6432 - loss: 0.9844 - val_accuracy: 0.5140 - val_loss: 1.3156
Epoch 7/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 31s 84ms/step - accuracy: 0.6954 - loss: 0.8511 - val_accuracy: 0.5229 - val_loss: 1.3172
Epoch 8/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 41s 84ms/step - accuracy: 0.7446 - loss: 0.7232 - 

# Define input size
input_shape = (128, 128, 1)

In [8]:
# Define input size
input_shape = (128, 128, 1)

total_images = 0
for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        count = len(os.listdir(os.path.join(DATA_PATH, dir_)))
        print(f"{dir_} has {count} number of images")
        total_images += count

print(f"\ntotal images are {total_images}")

img_arr = []
img_label = []
label_to_text = {}
label = 0

for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        for f in os.listdir(os.path.join(DATA_PATH, dir_)):
            img = cv2.imread(os.path.join(DATA_PATH, dir_, f), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, input_shape[:2])  # Resize the image
            img_arr.append(np.expand_dims(img, axis=-1))
            img_label.append(label)
        print(f"loaded {dir_} images to numpy arrays...")
        label_to_text[label] = dir_
        label += 1

img_arr = np.array(img_arr)
img_label = np.array(img_label)
img_label = OneHotEncoder().fit_transform(img_label.reshape(-1, 1)).toarray()  # Convert to dense array

surprise has 1851 number of images
fear has 1839 number of images
neutral has 1880 number of images
sad has 1821 number of images
disgust has 1740 number of images
contempt has 1833 number of images
happy has 1862 number of images
anger has 1822 number of images

total images are 14648
loaded surprise images to numpy arrays...
loaded fear images to numpy arrays...
loaded neutral images to numpy arrays...
loaded sad images to numpy arrays...
loaded disgust images to numpy arrays...
loaded contempt images to numpy arrays...
loaded happy images to numpy arrays...
loaded anger images to numpy arrays...


In [9]:
# Normalize pixel values
img_arr = img_arr / 255.

X_train, X_test, y_train, y_test = train_test_split(img_arr, img_label,
                                                    shuffle=True, stratify=img_label,
                                                    train_size=TRAIN_SIZE, random_state=42)

def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [10]:
# Train and evaluate the model
print(f"Training model with input shape: {input_shape}")
model = create_cnn_model(input_shape, NUM_CLASSES)
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Training model with input shape: (128, 128, 1)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 155s 419ms/step - accuracy: 0.2503 - loss: 1.9219 - val_accuracy: 0.4307 - val_loss: 1.4608
Epoch 2/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 190s 388ms/step - accuracy: 0.4555 - loss: 1.4173 - val_accuracy: 0.4894 - val_loss: 1.3466
Epoch 3/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 139s 379ms/step - accuracy: 0.5463 - loss: 1.1906 - val_accuracy: 0.5075 - val_loss: 1.3165
Epoch 4/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 144s 385ms/step - accuracy: 0.6253 - loss: 1.0342 - val_accuracy: 0.5276 - val_loss: 1.3269
Epoch 5/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 156s 425ms/step - accuracy: 0.7085 - loss: 0.8138 - val_accuracy: 0.5491 - val_loss: 1.3484
Epoch 6/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 184s 378ms/step - accuracy: 0.7966 - loss: 0.5939 - val_accuracy: 0.5546 - val_loss: 1.4265
Epoch 7/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 141s 376ms/step - accuracy: 0.8693 - loss: 0.3977 - val_accuracy: 0.5686 - val_loss: 1.6014
Epoch 8/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 139s 379ms/step - accuracy: 0.9190 -

# Define input size
input_shape = (256, 256, 1)

In [11]:
# Define input size
input_shape = (256, 256, 1)

total_images = 0
for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        count = len(os.listdir(os.path.join(DATA_PATH, dir_)))
        print(f"{dir_} has {count} number of images")
        total_images += count

print(f"\ntotal images are {total_images}")

img_arr = []
img_label = []
label_to_text = {}
label = 0

for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        for f in os.listdir(os.path.join(DATA_PATH, dir_)):
            img = cv2.imread(os.path.join(DATA_PATH, dir_, f), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, input_shape[:2])  # Resize the image
            img_arr.append(np.expand_dims(img, axis=-1))
            img_label.append(label)
        print(f"loaded {dir_} images to numpy arrays...")
        label_to_text[label] = dir_
        label += 1

img_arr = np.array(img_arr)
img_label = np.array(img_label)
img_label = OneHotEncoder().fit_transform(img_label.reshape(-1, 1)).toarray()  # Convert to dense array

surprise has 1851 number of images
fear has 1839 number of images
neutral has 1880 number of images
sad has 1821 number of images
disgust has 1740 number of images
contempt has 1833 number of images
happy has 1862 number of images
anger has 1822 number of images

total images are 14648
loaded surprise images to numpy arrays...
loaded fear images to numpy arrays...
loaded neutral images to numpy arrays...
loaded sad images to numpy arrays...
loaded disgust images to numpy arrays...
loaded contempt images to numpy arrays...
loaded happy images to numpy arrays...
loaded anger images to numpy arrays...


In [12]:
# Normalize pixel values
img_arr = img_arr / 255.

X_train, X_test, y_train, y_test = train_test_split(img_arr, img_label,
                                                    shuffle=True, stratify=img_label,
                                                    train_size=TRAIN_SIZE, random_state=42)

def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [13]:
# Train and evaluate the model
print(f"Training model with input shape: {input_shape}")
model = create_cnn_model(input_shape, NUM_CLASSES)
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Training model with input shape: (256, 256, 1)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 697s 2s/step - accuracy: 0.2478 - loss: 2.1886 - val_accuracy: 0.4283 - val_loss: 1.4926
Epoch 2/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 709s 2s/step - accuracy: 0.4956 - loss: 1.3495 - val_accuracy: 0.5027 - val_loss: 1.3582
Epoch 3/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 669s 2s/step - accuracy: 0.6484 - loss: 1.0028 - val_accuracy: 0.5157 - val_loss: 1.3612
Epoch 4/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 669s 2s/step - accuracy: 0.7911 - loss: 0.6145 - val_accuracy: 0.5386 - val_loss: 1.5398
Epoch 5/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 680s 2s/step - accuracy: 0.9087 - loss: 0.3026 - val_accuracy: 0.5597 - val_loss: 1.9340
Epoch 6/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 672s 2s/step - accuracy: 0.9646 - loss: 0.1383 - val_accuracy: 0.5573 - val_loss: 2.4590
Epoch 7/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 672s 2s/step - accuracy: 0.9837 - loss: 0.0750 - val_accuracy: 0.5567 - val_loss: 2.5719
Epoch 8/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 680s 2s/step - accuracy: 0.9913 - loss: 0.0511 - val_accu